In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
crr_dir = os.getcwd()
proj_dir = crr_dir[0 : -10]
print('Working directory: ', proj_dir)

Working directory:  /home/jh/2python_pr/Abilene_DRL/


In [3]:
# organize node and link features
f = open(proj_dir + 'data/topo-2003-04-10.txt', 'r')
node_feature = np.empty((0, 4))
link_feature = np.empty((0, 4))

# extract node and link feature from topo-2003-04-10.txt
for i, each_line in enumerate(f):

    if i >=2 and i < 14:
        each_line_data = each_line.split('\t')
        each_line_data[-1] = each_line_data[-1][0:-2]
        node_feature = np.concat((node_feature, [each_line_data]))
    if i >= 18:
        each_line_data = each_line.split('\t')
        each_line_data[-1] = each_line_data[-1][0:-1]
        link_feature = np.concat((link_feature, [each_line_data]))

# create node index
id_arr = np.arange(node_feature.shape[0]).reshape(-1, 1)
node_feature = np.concat((id_arr, node_feature), axis = 1)

f.close()

In [4]:
f_l = open(proj_dir + 'data/links', 'r')

# extract link index from links file
link_info = np.empty((0, 4))
for i, each_line in enumerate(f_l):
    if i >= 2:
        st = each_line.split(',')[0]
        dt = each_line.split(',')[1].split(' ')
        link_type = dt[-1][0]
        crr_list = [st]
        crr_list.extend(dt[0:2])
        crr_list.append(link_type)
        link_info = np.concat((link_info, [crr_list]))
f_l.close()

In [5]:
link_index = np.empty((0, 1), dtype=int)
for i in range(len(link_feature)):
    for j in range(len(link_info)):       
        if np.all(link_feature[i, 0:2] == link_info[j, 0:2]):
            link_index = np.concat((link_index, np.reshape(link_info[j, 2], (-1,1)).astype(int)-1 ))
link_feature = np.concat((link_feature, link_index), axis = 1)

In [6]:
# In link feature, define source and destination id from node feature
num_row = link_feature.shape[0]
sd_id = np.empty((num_row, 2), dtype = 'int')
for row in range(num_row):
    source = link_feature[row, 0]
    destination = link_feature[row, 1]
    s_id = node_feature[node_feature[:, 1] == source, 0]
    d_id = node_feature[node_feature[:, 1] == destination, 0]
    sd_id[row, 0] = int(s_id[0])
    sd_id[row, 1] = int(d_id[0])
link_feature = np.concat((link_feature, sd_id), axis = 1)
link_feature = link_feature[:, [0, 1, 5, 6, 4, 2, 3]]

In [7]:
with open(proj_dir + 'data_processed/node_list_data.npy', 'wb') as f:
    header = np.save(f, 'node info: [id, name, city, latitude, longitude], link info: [start, desination, start_id, destination_id, link_id, capacity (kbps), OSPF, link_index]')
    node = np.save(f, node_feature)
    link = np.save(f, link_feature)